Jigsaw toxic comment challenge using LSTM Cells

In [23]:
import os 
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense,Input,GlobalMaxPooling1D
from keras.layers import LSTM,Embedding,SpatialDropout1D,Bidirectional
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score

In [4]:
max_seq_len = 100
max_vocab_size = 20000
embedding_dim = 100
batch_size = 128
epochs = 5
val_split = 0.3

In [5]:
#loading wordvecs from glove
word2vec = {}
with open(os.path.join('F:/my files/python files/neural networks/keras projects/data/glove/glove.6B.%sd.txt' %embedding_dim), 
          encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('no of word vectors:', len(word2vec))        
        

no of word vectors: 400000


In [7]:
#loading the data
data = pd.read_csv('F:/my files/python files/neural networks/keras projects/dataset/toxic-comments/train.csv')
comments = data['comment_text']
comments = comments.fillna('NIL')
comments = comments.values
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
targets = data[labels]
targets = targets.values


In [8]:
#tokenisation
tokens = Tokenizer(num_words=max_vocab_size)
tokens.fit_on_texts(comments)
seq = tokens.texts_to_sequences(comments)
print(seq[2])

[412, 437, 73, 134, 14, 249, 2, 71, 314, 78, 50, 9, 13, 626, 8, 2284, 492, 502, 102, 4, 611, 2, 35, 325, 126, 363, 3, 29, 38, 27, 52, 208, 2, 434, 57, 36, 1, 2394, 93, 1, 737, 468]


In [10]:
word2idx = tokens.word_index
print('unique words:', len(word2idx))
print(word2idx['is'])


unique words: 210337
8


In [11]:
pad_data = pad_sequences(seq, maxlen=max_seq_len, truncating='pre')
print(pad_data[2])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0  412  437
   73  134   14  249    2   71  314   78   50    9   13  626    8 2284  492
  502  102    4  611    2   35  325  126  363    3   29   38   27   52  208
    2  434   57   36    1 2394   93    1  737  468]


In [14]:
num_words = min(max_vocab_size, len(word2idx)+1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word2idx.items():
    if i < max_vocab_size:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
embedding_layer = Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_seq_len, trainable=False)        
        

In [17]:
#neural network
inp = Input(shape=(max_seq_len,))
x = embedding_layer(inp)
x = Bidirectional(LSTM(15, return_sequences=True))(x)
x = GlobalMaxPooling1D()(x)
out = Dense(len(labels), activation='sigmoid')(x)




In [18]:
model = Model(inp, out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 30)           13920     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 186       
Total params: 2,014,106
Trainable params: 14,106
Non-trainable params: 2,000,000
_________________________________________________________________


In [21]:
checkpoint = ModelCheckpoint(filepath='comm_lstm.weights.best.hdf5', verbose=1, save_best_only=True)
model.fit(pad_data, targets, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_split=val_split, verbose=1)

Train on 111699 samples, validate on 47872 samples


Epoch 1/5


   128/111699 [..............................] - ETA: 7:46:02 - loss: 0.0398 - acc: 0.9844

   256/111699 [..............................] - ETA: 3:54:43 - loss: 0.0360 - acc: 0.9883

   384/111699 [..............................] - ETA: 2:38:03 - loss: 0.0356 - acc: 0.9900

...

111360/111699 [============================>.] - ETA: 0s - loss: 0.0453 - acc: 0.9835

111488/111699 [============================>.] - ETA: 0s - loss: 0.0453 - acc: 0.9835

111616/111699 [============================>.] - ETA: 0s - loss: 0.0453 - acc: 0.9835

111699/111699 [==============================] - 246s 2ms/step - loss: 0.0453 - acc: 0.9835 - val_loss: 0.0490 - val_acc: 0.9824



Epoch 00005: val_loss did not improve


In [25]:
y_hat = model.predict(pad_data)
auc = roc_auc_score(targets, y_hat) * 100
print(auc)

98.0806951621
